In [9]:
#API Keys
CLIENT_ID = 'lpakYt45RJlvK6Lp6ybehw'
SECRET_KEY = 'XXHGvxgxAhlEHK6-KMy8nP_QmcedXg'

In [10]:
#imports the request module and acquires an access token from the api by giving it our user name and password for reddit which we can then use to access and use the API
import requests

auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)
data = {
    'grant_type' : 'password',
    'username' : 'PiccoloNo3695',
    'password' : 'scooby2203'
}

headers = {'User-Agent' : 'MyAPI/0.0.1'}
res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
TOKEN = res.json()['access_token']
headers['Authorization'] = f'bearer {TOKEN}'

In [11]:
#checks to make sure api is connecting properly, response 200 means all is well
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [12]:
#gets all the hot posts from the python subreddit use the paramas limit to change the amount of posts you are getting (100 is the max), also change the url at the end to either new/hot/top/rising depending on what you want from the subreddit
res = requests.get('http://oauth.reddit.com/r/python/top/?t=all', headers=headers, params={'limit' : '100'})
res.json()

#sets the variable after to equal the name of the last post in the first request
after = res.json()['data']['after']

In [ ]:
#this sorts through the request json, picking out the important information and converting it into a dataframe table format where it can be better interpreted
import pandas as pd

#df = pd.read_csv('Questions.csv')
df = pd.DataFrame()
df = df.loc[:, ~df.columns.str.contains('Unnamed')]
for post in res.json()['data']['children']:
    name = post ['data']['name']
    link = name[3:]
    df = df.append({
        'Question' : post['data']['selftext'],
        'QuestionAlternative' : post['data']['title'],
        'NFSW' : post['data']['over_18'],
        'Name' : link,
        'Source' : 'Reddit ' + "- r/" + post ['data']['subreddit'],
        'Number of Comments' : post['data']['num_comments']
    }, ignore_index=True)
df


In [ ]:
#this creates a new request which fetches all the posts after the last post in the first request and then adds those posts to the dataframe in the same format
#it then loops back adding +1 to the i variable and creating a new request that fetches all the posts after the last request made
#the while loop runs until i is above the value set in the while loop, so if the value is set to 5, 5 requests will be made ontop of the original request
#and then you will have 600 posts in the dataframe
i = 1

while i <= 100:

    i = i + 1

    res2 = requests.get('http://oauth.reddit.com/r/python/top/?t=all', headers=headers, params={'limit' : '100', 'after' : after})
    res2.json()

    #sets the variable after to equal the name of the last post in the last request made
    after = res2.json()['data']['after']

    for post in res2.json()['data']['children']:
        name2 = post ['data']['name']
        link2 = name2[3:]
        df = df.append({
            'Question' : post['data']['selftext'],
            'QuestionAlternative' : post['data']['title'],
            'NFSW' : post['data']['over_18'],
            'Name' : link2,
            'Source' : 'Reddit ' + "- r/" + post ['data']['subreddit'],
            'Number of Comments' : post['data']['num_comments']
        }, ignore_index=True)

#this prints the dataframe when the while loop as ended
df

In [15]:
#exports the dataframe to the Questions CSV file ready to be cleaned
df.to_csv('Questions.csv')